# Zout + zuur → pH-doel in de maisch (gekoppeld aan Henderson–Hasselbalch)

**Doel.** Dit notebook koppelt *waterbehandeling* (zouttoevoegingen + zuurcorrectie) aan een *vereenvoudigd mash pH-model* (Henderson–Hasselbalch, Notebook 1). Je verkent hoe (i) **ionen/alkaliniteit** en (ii) **zuurtoevoeging** samen de verwachte maisch-pH sturen.

> **Didactische waarschuwing (bewust).** Dit is **geen** volledige waterchemie- of moutbufferingssimulator. We gebruiken een *effectieve buffer* rond een gekozen pKₐ en een *koppelfactor* die de invloed van (residuele) alkaliniteit op de start-pH van de maisch samenvat. Het doel is *inzicht* en *systematisch redeneren*, niet lab-grade voorspelling.

## Overzicht
1. Theorie (kort)
2. Parameters
3. Zouten → nieuw waterprofiel + RA
4. RA + zuur → voorspelde mash pH (HH-model)
5. Simulatie & plots
6. Mini-optimizer: zoek zuur (en optioneel zouten) voor een pH-doel
7. Interpretatievragen


## 1. Theorie (kort)

### 1.1 Henderson–Hasselbalch als *effectieve buffer*
Voor één bufferpaar (HA/A⁻):

$$ \mathrm{pH} = \mathrm{p}K_a + \log_{10}\left(\frac{[A^-]}{[HA]}\right) $$

We modelleren de maisch als één *effectief* bufferpaar met:
- $\mathrm{p}K_{a,\,eff}$ (dominante buffer rond het relevante pH-bereik)
- $C_T = [HA] + [A^-]$ (totale effectieve buffercapaciteit)
- een start-pH $\mathrm{pH}_{0,\,eff}$ vóór zuurcorrectie.

### 1.2 Residual Alkalinity (RA) als pH-'duwkracht'
Een veelgebruikte (vereenvoudigde) vuistregel in brouwen:

$$ RA_{(as\ CaCO_3)} \approx Alkalinity_{(as\ CaCO_3)} - \frac{Ca}{3.5} - \frac{Mg}{7} $$

met Ca en Mg in **mg/L** en RA in **mg/L als CaCO₃**.

**Koppeling naar mash pH.** We vatten de invloed van RA op de *start-pH* van de maisch samen met een instelbare koppelfactor:

$$ \mathrm{pH}_{0,\,eff} = \mathrm{pH}_{0,\,base} + k_{RA}\,\frac{(RA-RA_{ref})}{100} $$

- $k_{RA}$: pH-eenheden per 100 mg/L (ppm) RA-as-CaCO₃ (typisch klein; hier didactisch instelbaar).
- $RA_{ref}$: referentiepunt (vaak 0).

### 1.3 Zouttoevoegingen en zuur
- Zouten wijzigen ionen (Ca, Mg, Na, Cl, SO₄, HCO₃) en dus indirect RA.
- Zuurtoevoeging levert $H^+$ (equivalenten) die A⁻ omzet in HA: $A^- + H^+ \rightarrow HA$.


In [ ]:
# --- Imports ---
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# =========================================================
# 2. PARAMETERS (pas deze aan)
# =========================================================

# ---- Batch/volume ----
V_l = 20.0  # [L] effectief maischwatervolume (vloeibare fase)

# ---- Startwater (mg/L) ----
# Vul in op basis van wateranalyse. HCO3 kan ook via alkaliniteit (as CaCO3).
water_mgL = {
    'Ca': 30.0,
    'Mg': 8.0,
    'Na': 12.0,
    'Cl': 25.0,
    'SO4': 35.0,
    'HCO3': 120.0,  # mg/L
}

# ---- Henderson–Hasselbalch 'effectieve buffer' (Notebook 1-achtig) ----
pH0_base = 5.60     # [-] start-pH van de maisch bij RA_ref (didactische basiswaarde)
pKa_eff = 6.80      # [-] effectieve pKa rond dit pH-bereik
CT_mM  = 25.0       # [mM] totale effectieve bufferconcentratie (HA + A-)

# ---- Koppeling RA -> pH0_eff ----
RA_ref = 0.0        # [mg/L as CaCO3]
k_RA   = 0.10       # [pH / (100 mg/L as CaCO3)]  (instelbaar)

# ---- Zouttoevoegingen (gram per batch) ----
# Zet op 0.0 als je een zout niet gebruikt.
salts_g = {
    'CaCl2_2H2O': 0.0,   # calciumchloride dihydraat
    'CaSO4_2H2O': 0.0,   # gips
    'MgSO4_7H2O': 0.0,   # epsom
    'NaCl': 0.0,
    'NaHCO3': 0.0,       # baksoda
}

# ---- Zuurcorrectie ----
acid = {
    'type': 'lactic_88',   # 'lactic_88' of 'phosphoric_85'
    'mL': 0.0,
}

# ---- Doel (voor optimizer) ----
pH_target = 5.35

print('Parameters geladen.')


In [ ]:
# =========================================================
# 3. HULPFUNCTIES: waterchemie (vereenvoudigd)
# =========================================================

MOLAR_MASS = {
    'CaCl2_2H2O': 147.02,
    'CaSO4_2H2O': 172.17,
    'MgSO4_7H2O': 246.47,
    'NaCl': 58.44,
    'NaHCO3': 84.01,
}

# Ionenbijdragen per mol zout (stoichiometrie)
SALT_STOICH = {
    'CaCl2_2H2O': {'Ca': 1, 'Cl': 2},
    'CaSO4_2H2O': {'Ca': 1, 'SO4': 1},
    'MgSO4_7H2O': {'Mg': 1, 'SO4': 1},
    'NaCl': {'Na': 1, 'Cl': 1},
    'NaHCO3': {'Na': 1, 'HCO3': 1},
}

ION_MASS = {
    'Ca': 40.078,
    'Mg': 24.305,
    'Na': 22.990,
    'Cl': 35.453,
    'SO4': 96.06,
    'HCO3': 61.016,
}

def add_salts_to_water_mgL(water_mgL, salts_g, V_l):
    """Return nieuw waterprofiel (mg/L) na zouttoevoegingen aan volume V_l."""
    new = dict(water_mgL)
    for salt, g in salts_g.items():
        if g is None or g == 0:
            continue
        if salt not in MOLAR_MASS:
            raise ValueError(f'Onbekend zout: {salt}')
        mol = g / MOLAR_MASS[salt]
        for ion, nu in SALT_STOICH[salt].items():
            added_mg = mol * nu * ION_MASS[ion] * 1000.0  # g->mg
            added_mgL = added_mg / V_l
            new[ion] = new.get(ion, 0.0) + added_mgL
    return new

def alkalinity_as_CaCO3_from_HCO3(hco3_mgL):
    """Converteer HCO3- (mg/L) naar alkaliniteit als CaCO3 (mg/L)."""
    return hco3_mgL * 50.0 / 61.016

def residual_alkalinity_as_CaCO3(water_mgL):
    """RA (mg/L as CaCO3) volgens vuistregel."""
    Alk = alkalinity_as_CaCO3_from_HCO3(water_mgL.get('HCO3', 0.0))
    Ca = water_mgL.get('Ca', 0.0)
    Mg = water_mgL.get('Mg', 0.0)
    RA = Alk - (Ca / 3.5) - (Mg / 7.0)
    return RA, Alk

def water_summary(w):
    keys = ['Ca','Mg','Na','Cl','SO4','HCO3']
    return {k: float(w.get(k,0.0)) for k in keys}


In [ ]:
# =========================================================
# 4. HULPFUNCTIES: mash pH (HH) + zuur
# =========================================================

ACID_DB = {
    # benaderingen; voldoende voor onderwijsmodel
    # 'H_per_mL' = mol H+ equivalent per mL zuur
    'lactic_88': {
        'label': 'Melkzuur 88%',
        'mw': 90.078,
        'density_g_per_mL': 1.20,
        'mass_fraction': 0.88,
        'equiv_per_mol': 1.0,
    },
    'phosphoric_85': {
        'label': 'Fosforzuur 85% (1e dissociatie als effectief)',
        'mw': 97.995,
        'density_g_per_mL': 1.685,
        'mass_fraction': 0.85,
        'equiv_per_mol': 1.0,  # didactisch: enkel 1e proton als sterk effectief in mash-bereik
    },
}

def acid_H_mol_per_mL(acid_type):
    a = ACID_DB[acid_type]
    g_pure_per_mL = a['density_g_per_mL'] * a['mass_fraction']
    mol_per_mL = g_pure_per_mL / a['mw']
    return mol_per_mL * a['equiv_per_mol']

def hh_pH_after_acid(pH0, pKa, CT_mM, V_l, nH_mol):
    """HH-model: buffer HA/A- met totale concentratie CT_mM (mM). Zuur zet A- om naar HA."""
    CT_mol = (CT_mM/1000.0) * V_l  # mol
    # Startverhouding r = [A-]/[HA]
    r0 = 10**(pH0 - pKa)
    A0 = CT_mol * (r0/(1+r0))
    HA0 = CT_mol * (1/(1+r0))
    # Zuur consumeert A-
    d = min(nH_mol, A0)
    A1 = A0 - d
    HA1 = HA0 + d
    # Numerieke veiligheid
    A1 = max(A1, 1e-18)
    HA1 = max(HA1, 1e-18)
    pH1 = pKa + np.log10(A1/HA1)
    return float(pH1)

def pH0_from_RA(pH0_base, RA, RA_ref=0.0, k_RA=0.10):
    return float(pH0_base + k_RA*((RA-RA_ref)/100.0))


In [ ]:
# =========================================================
# 5. RUN: zouten + RA + zuur -> mash pH
# =========================================================

# Water vóór/na zout
w0 = water_summary(water_mgL)
w1 = water_summary(add_salts_to_water_mgL(w0, salts_g, V_l))

# RA vóór/na
RA0, Alk0 = residual_alkalinity_as_CaCO3(w0)
RA1, Alk1 = residual_alkalinity_as_CaCO3(w1)

# Start-pH (effectief) na zouten/RA
pH0_eff0 = pH0_from_RA(pH0_base, RA0, RA_ref, k_RA)
pH0_eff1 = pH0_from_RA(pH0_base, RA1, RA_ref, k_RA)

# Zuur -> H+ mol
H_per_mL = acid_H_mol_per_mL(acid['type'])
nH = acid['mL'] * H_per_mL

# pH voorspelling (met en zonder zouten)
pH_pred0 = hh_pH_after_acid(pH0_eff0, pKa_eff, CT_mM, V_l, nH)
pH_pred1 = hh_pH_after_acid(pH0_eff1, pKa_eff, CT_mM, V_l, nH)

print('--- Waterprofiel (mg/L) ---')
print('Start:', w0)
print('Na zouten:', w1)
print('\n--- Alkaliniteit / RA (mg/L as CaCO3) ---')
print(f'Alk start: {Alk0:6.1f} | RA start: {RA0:6.1f}')
print(f'Alk na zout: {Alk1:6.1f} | RA na zout: {RA1:6.1f}')
print('\n--- Mash pH (HH-model) ---')
print(f'pH0_eff (startwater): {pH0_eff0:.3f} -> pH na zuur: {pH_pred0:.3f}')
print(f'pH0_eff (na zouten): {pH0_eff1:.3f} -> pH na zuur: {pH_pred1:.3f}')


In [ ]:
# =========================================================
# 6. PLOTS
# =========================================================

def plot_ion_profile(before, after, title='Ionprofiel (mg/L)'):
    ions = ['Ca','Mg','Na','Cl','SO4','HCO3']
    b = np.array([before.get(i,0.0) for i in ions], dtype=float)
    a = np.array([after.get(i,0.0) for i in ions], dtype=float)
    x = np.arange(len(ions))
    width = 0.38
    plt.figure(figsize=(8,4))
    plt.bar(x - width/2, b, width, label='Start')
    plt.bar(x + width/2, a, width, label='Na zouten')
    plt.xticks(x, ions)
    plt.ylabel('mg/L')
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.show()

plot_ion_profile(w0, w1)

# pH versus zuurvolume
mL_grid = np.linspace(0, 10, 101)
pH_curve = [hh_pH_after_acid(pH0_eff1, pKa_eff, CT_mM, V_l, m*H_per_mL) for m in mL_grid]

plt.figure(figsize=(8,4))
plt.plot(mL_grid, pH_curve)
plt.axhline(pH_target, linestyle='--', linewidth=1)
plt.xlabel(f"{ACID_DB[acid['type']]['label']} (mL)")
plt.ylabel('Voorspelde mash pH')
plt.title('Zuurcorrectie → mash pH (na zouten)')
plt.tight_layout()
plt.show()


## 7. Mini-optimizer
Zoek de **zuurhoeveelheid** (en optioneel beperkte zoutvariatie) die het dichtst bij de doel-pH komt.

- Standaard optimaliseren we enkel **zuur (mL)**.
- Optioneel kan je ook **CaCl₂** en **CaSO₄** in kleine intervallen laten meescannen.


In [ ]:
# =========================================================
# 7. MINI-OPTIMIZER
# =========================================================

def optimize_acid_only(water_mgL, salts_g, V_l, pH_target,
                      pH0_base, pKa_eff, CT_mM, RA_ref, k_RA,
                      acid_type='lactic_88', mL_max=12.0, n=241):
    w = add_salts_to_water_mgL(water_mgL, salts_g, V_l)
    RA, _ = residual_alkalinity_as_CaCO3(w)
    pH0_eff = pH0_from_RA(pH0_base, RA, RA_ref, k_RA)
    HmL = acid_H_mol_per_mL(acid_type)
    mL_grid = np.linspace(0, mL_max, n)
    pH_grid = np.array([hh_pH_after_acid(pH0_eff, pKa_eff, CT_mM, V_l, m*HmL) for m in mL_grid])
    idx = int(np.argmin(np.abs(pH_grid - pH_target)))
    return {
        'best_mL': float(mL_grid[idx]),
        'best_pH': float(pH_grid[idx]),
        'pH0_eff': float(pH0_eff),
        'RA': float(RA),
        'curve_mL': mL_grid,
        'curve_pH': pH_grid,
    }

res = optimize_acid_only(w0, salts_g, V_l, pH_target,
                        pH0_base, pKa_eff, CT_mM, RA_ref, k_RA,
                        acid_type=acid['type'])

print('--- Optimizer (zuur only) ---')
print(f"RA na zouten: {res['RA']:.1f} mg/L as CaCO3")
print(f"pH0_eff:      {res['pH0_eff']:.3f}")
print(f"Doel pH:      {pH_target:.3f}")
print(f"Beste zuur:   {res['best_mL']:.2f} mL ({ACID_DB[acid['type']]['label']})")
print(f"Voorspelde pH:{res['best_pH']:.3f}")

plt.figure(figsize=(8,4))
plt.plot(res['curve_mL'], res['curve_pH'])
plt.axhline(pH_target, linestyle='--', linewidth=1)
plt.axvline(res['best_mL'], linestyle='--', linewidth=1)
plt.xlabel(f"{ACID_DB[acid['type']]['label']} (mL)")
plt.ylabel('Voorspelde mash pH')
plt.title('Optimizer: zuur → pH-doel')
plt.tight_layout()
plt.show()


## 8. Interpretatievragen
1. Welke parameter beïnvloedt de voorspelde pH het sterkst: **$C_T$**, **$pK_{a,eff}$**, **$k_{RA}$**, of het **startwaterprofiel**? Licht toe.
2. Waarom is het riskant om RA rechtstreeks als pH-voorspeller te gebruiken zonder rekening te houden met **moutbuffering**?
3. Kies twee oplossingen die dezelfde pH opleveren maar een andere **Cl/SO₄**-balans. Wat verwacht je sensorisch (conceptueel)?
4. Welke aannames zouden als eerste falen bij (i) zeer donkere stort, (ii) hoge ionic strength, (iii) grote zuurtoevoegingen?
5. Hoe zou je dit model experimenteel kunnen kalibreren voor één brouwhuis (meetplan)?

**Kernboodschap.** pH-correctie is slechts één doel; waterbehandeling is een *multi-doel* ontwerp (pH, smaakbalans, ionlimieten). Dit notebook helpt die doelen systematisch te koppelen.
